In [4]:
import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
import torch
from torchvision import transforms
from skimage import img_as_ubyte

# Function to calculate PSNR
def psnr(target, ref):
    mse = np.mean((target - ref) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    return 20 * np.log10(max_pixel / np.sqrt(mse))

# Function to calculate SSIM
def calculate_ssim(target, ref):
    return ssim(target, ref, multichannel=True)

# Load model from torchhub
preprocess, postprocess, model = torch.hub.load('tnwei/waternet', 'waternet')
model.eval()



Using cache found in /root/.cache/torch/hub/tnwei_waternet_main


WaterNet(
  (cmg): ConfidenceMapGenerator(
    (conv1): Conv2d(12, 128, kernel_size=(7, 7), stride=(1, 1), padding=same)
    (relu1): ReLU()
    (conv2): Conv2d(128, 128, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (relu2): ReLU()
    (conv3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (relu3): ReLU()
    (conv4): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (relu4): ReLU()
    (conv5): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=same)
    (relu5): ReLU()
    (conv6): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=same)
    (relu6): ReLU()
    (conv7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (relu7): ReLU()
    (conv8): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (sigmoid): Sigmoid()
  )
  (wb_refiner): Refiner(
    (conv1): Conv2d(6, 32, kernel_size=(7, 7), stride=(1, 1), padding=same)
    (conv2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1), pad

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Input and output directories
input_folder = "/content/drive/MyDrive/aqua_data/invasive"
output_folder = "/content/drive/MyDrive/SRCNN/water/inv"

# Ensure output directory exists
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each image in the directory
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check for image files
        file_path = os.path.join(input_folder, filename)
        im = cv2.imread(file_path)
        if im is None:
            continue  # Skip files that can't be opened

        # Preprocess the image
        rgb_im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        rgb_im = cv2.resize(rgb_im, (720, 480))

        # Inference
        rgb_ten, wb_ten, he_ten, gc_ten = preprocess(rgb_im)
        out_ten = model(rgb_ten, wb_ten, he_ten, gc_ten)
        out_im = postprocess(out_ten)[0]

        # Convert tensor to numpy array for evaluation
        rgb_im_np = np.array(rgb_im)
        out_im_np = np.array(out_im)

        # Calculate PSNR, MSE, and SSIM
        psnr_value = psnr(rgb_im_np, out_im_np)
        mse_value = np.mean((rgb_im_np - out_im_np) ** 2)
        ssim_value = calculate_ssim(rgb_im_np, out_im_np)

        # Print metrics
        print(f"PSNR: {psnr_value}, MSE: {mse_value}, SSIM: {ssim_value}")

        # Save the processed image
        output_file_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_file_path, cv2.cvtColor(out_im_np, cv2.COLOR_RGB2BGR))


<ipython-input-4-4595a9bfc546>:19: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  return ssim(target, ref, multichannel=True)


PSNR: 28.749148541044832, MSE: 86.7292332175926, SSIM: 0.9377677519361288
PSNR: 27.502386763506472, MSE: 115.56908757716049, SSIM: 0.7529845826101426
PSNR: 29.531863336565507, MSE: 72.42592110339505, SSIM: 0.9760799593783833
PSNR: 28.338349541394212, MSE: 95.33347125771606, SSIM: 0.9367452694328433
PSNR: 28.450368321684117, MSE: 92.90595003858024, SSIM: 0.9394992709549198
PSNR: 28.792128157222248, MSE: 85.87515721450617, SSIM: 0.9647001438802248
PSNR: 27.519659132337388, MSE: 115.11036940586419, SSIM: 0.746435922103515
PSNR: 27.91128958890208, MSE: 105.18441647376544, SSIM: 0.8817263146320112
PSNR: 29.4749801828161, MSE: 73.38078317901234, SSIM: 0.9760686773292013
PSNR: 28.89193313314227, MSE: 83.924168595679, SSIM: 0.9372407736399663
PSNR: 28.609181654551804, MSE: 89.569921875, SSIM: 0.9400851762253103
PSNR: 27.68544743926318, MSE: 110.79894290123457, SSIM: 0.839791561121008
PSNR: 28.304313448813183, MSE: 96.08354456018519, SSIM: 0.9388148093506147
PSNR: 28.43960407848472, MSE: 93.136